# Utilizing Open LLMs with the OpenAI API
This Notebook uses the same basic OpenAI Python API to run different local and serverless models

### Notebook Setup

In [13]:
# %pip install openai langchain langchain_openai

In [14]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Set model variables
OPENAI_BASE_URL = "https://api.openai.com/v1"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION")

OLLAMA_BASE_URL = "http://localhost:11434/v1"
OLLAMA_API_KEY = "ollama"

TOGETHER_BASE_URL = "https://api.together.xyz"
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

### Initialize tools and models for all demos

In [15]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        },
    },
]

MESSAGES = [
    {
        "role": "system",
        "content": "You are a helpful assistant that can access external functions. Please provide responses based on the information from these function calls.",
    },
    {
        "role": "user",
        "content": "What is the current temperature of New York, San Francisco and Chicago?",
    },
]

### Run a serverless Mixtral with the OpenAI API

In [25]:
import json
from openai import OpenAI

client = OpenAI(
    base_url=TOGETHER_BASE_URL,
    api_key=TOGETHER_API_KEY,
)


response = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    messages=MESSAGES,
    tools=TOOLS,
    tool_choice="auto",
)

print(json.dumps(response.choices[0].message.model_dump()["tool_calls"], indent=2))

[
  {
    "id": "call_aohq18yhi3yav8p7z81jyjhw",
    "function": {
      "arguments": "{\"location\":\"New York, NY\",\"format\":\"fahrenheit\"}",
      "name": "get_current_weather"
    },
    "type": "function"
  },
  {
    "id": "call_2wbb93smogfhtoiq5gcac2pw",
    "function": {
      "arguments": "{\"location\":\"San Francisco, CA\",\"format\":\"fahrenheit\"}",
      "name": "get_current_weather"
    },
    "type": "function"
  },
  {
    "id": "call_3zkezuh2293wzjjiqwn1g63d",
    "function": {
      "arguments": "{\"location\":\"Chicago, IL\",\"format\":\"fahrenheit\"}",
      "name": "get_current_weather"
    },
    "type": "function"
  }
]


#### You can even stream responses

In [26]:
stream = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    messages=MESSAGES,
    stream=True,
    tools=TOOLS,
    tool_choice="auto",
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="", flush=True)

 [
  {
    "name": "get_current_weather",
    "arguments": {
      "location": "New York, NY",
      "format": "fahrenheit"
    }
  },
  {
    "name": "get_current_weather",
    "arguments": {
      "location": "San Francisco, CA",
      "format": "fahrenheit"
    }
  },
  {
    "name": "get_current_weather",
    "arguments": {
      "location": "Chicago, IL",
      "format": "fahrenheit"
    }
  }
]

### Run 5 different models with the same API: 2 Serverless, 2 Local, 1 OpenAI

In [18]:
MODEL_CONFIGS = [
    {
        "description": "Llama2 Locally",
        "base_url": OLLAMA_BASE_URL,
        "api_key": OLLAMA_API_KEY,
        "model": "llama2",
    },
    {
        "description": "Mistral Locally",
        "base_url": OLLAMA_BASE_URL,
        "api_key": OLLAMA_API_KEY,
        "model": "mistral",
    },
    {
        "description": "Llama2 Serverless",
        "base_url": TOGETHER_BASE_URL,
        "api_key": TOGETHER_API_KEY,
        "model": "togethercomputer/llama-2-13b-chat",
    },
    {
        "description": "Mixtral Serverless",
        "base_url": TOGETHER_BASE_URL,
        "api_key": TOGETHER_API_KEY,
        "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
    },
    {
        "description": "OpenAI Serverless",
        "base_url": OPENAI_BASE_URL,
        "api_key": OPENAI_API_KEY,
        "model": "gpt-3.5-turbo-1106",
    },
]

MESSAGES_FOR_POEMS = [
    {
        "role": "system",
        "content": "You are a poet who writes whimsical haikus about a given topic.",
    },
    {
        "role": "user",
        "content": "Topic: Ducks",
    },
]

In [27]:
for model in MODEL_CONFIGS:

    print(f"Running {model['description']}:")
    print(f"-----------------------------\n")

    client = OpenAI(
        base_url=model["base_url"],
        api_key=model["api_key"],
    )

    response = client.chat.completions.create(
        model=model["model"],
        messages=MESSAGES_FOR_POEMS,
    )

    print(f"{response.choices[0].message.content}\n\n")

Running Llama2 Locally:
-----------------------------

Quacking in the pond,
Feathers fluffed, beaks dipped,
Ducks swim and play.

Ribbons of blue and gold,
Flapping wings, a gentle sight,
Ducks take flight.

In the sun or rain,
Quacking ducks bring joy again,
Waterfowl delight.

Splashes in the water,
Feathers flung, a ducky dance,
Happiness abounds.


Running Mistral Locally:
-----------------------------

 Quiet pond reflects,
Beneath, ducks weave graceful patterns,
Nature's art unfolds.

Dancing in the rain,
Ducks splash and play, pure joy abounds,
Life's simple pleasures.

Morning sun illumines,
Feathered friends paddle calmly by,
Serenity reigns.

Beneath the autumn sky,
Ducks align in perfect rows,
Autumn's graceful grace.


Running Llama2 Serverless:
-----------------------------

 Quacking in the pond,
Feathers fluffed and shining bright,
Ducks dance in the sun.


Running Mixtral Serverless:
-----------------------------

Feathers, bright and dappled,
Water's surface, gently b

### This also works well with LangChain

In [28]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


for model in MODEL_CONFIGS:

    print(f"Running {model['description']}:")
    print(f"-----------------------------\n")

    llm = ChatOpenAI(
        openai_api_base=model["base_url"],
        api_key=model["api_key"],
        model=model["model"],
        temperature=1.0,
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "Write a whimsical haiku about a given topic."),
            ("user", "Topic: Ducks"),
        ]
    )

    poem_runnable = prompt | llm

    response = poem_runnable.invoke({})
    print(f"{response.content}\n\n")

Running Llama2 Locally:
-----------------------------


Quacking echoes flow
Through the pond's reflective depths
Ducks dance with grace


Running Mistral Locally:
-----------------------------

 Quilt of feathers swirl,
River's mirror dances bright,
Ducks paddle, twinkle, world.


Running Llama2 Serverless:
-----------------------------

 Sure! Here's a whimsical haiku about ducks:

Quacking in the pond,
Feathers fluffed and feisty,
Ducks dance on water.


Running Mixtral Serverless:
-----------------------------

Quacking in glee,
Feathers of rainbow hues,
Pond dance in delight.


Running OpenAI Serverless:
-----------------------------

Feathers float and glide
Quacking waddlers on the pond
Ducks bring joy inside


